In [1]:
# У меня был опыт в решении подобной задачей, и я применил бы на эту тот же способ, а именно, градиентный бустинг
# Суть метода заключается в том, что бы посмотреть на исторические данные и обучить модель предиктить следующие данные по LTV 

In [ ]:
import pandas as pd

# Допустим, df у нас уже есть из базы
# df: колонки — User_ID, Reg_Date, Date, Revenue, Country, Platform, Referral
df['User_Age_Day'] = (df['Date'] - df['Reg_Date']).dt.days
df = df[df['User_Age_Day'] <= 60]

# когортный LTV: группируем по дню жизни, когорте
ltv_df = df.groupby(['Reg_Date', 'User_Age_Day']).agg({'Revenue': 'mean'}).groupby(level=0).cumsum().reset_index()

# добавим дополнительные фичи к когортам (например, средняя платформа, страна и т.д.)
meta = df.groupby('Reg_Date').agg({
    'Platform': lambda x: x.mode()[0],
    'Country': lambda x: x.mode()[0],
    'Referral': lambda x: x.mode()[0]
}).reset_index()

ltv_df = ltv_df.merge(meta, on='Reg_Date')

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

# Feature Engineering
X = ltv_df[['User_Age_Day', 'Platform', 'Country', 'Referral']]
X = pd.get_dummies(X) 
y = ltv_df['Revenue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = GradientBoostingRegressor()
model.fit(X_train, y_train)

# Прогнозируем для новых когорт
future_days = pd.DataFrame({
    'User_Age_Day': list(range(1, 61)),
    'Platform': 'Android',
    'Country': 'US',
    'Referral': 'link001'
})
future_days = pd.get_dummies(future_days)
ltv_forecast = model.predict(future_days)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1, 61), ltv_forecast, label="ML forecast")
plt.xlabel('Day')
plt.ylabel('Cumulative LTV')
plt.legend()
plt.title('Прогноз LTV по дням')
plt.grid(True)